# README

This is an example to illustrate how measurement notebooks could look like.
Of course this is easy to convert to a script, if you prefer working with spyder, for example.

Some things are not in a state yet that we're fully happy with, they're annotated correspondingly.

**TODO:** the initialization could be in a separate script or notebook. That would be useful if multiple notebooks run measurements, for example.

# Initialization

## Imports

In [ ]:
%matplotlib inline

In [ ]:
from pprint import pprint
import time

import numpy as np
from matplotlib import pyplot as plt
import h5py


from qcodes import Instrument, Station, find_or_create_instrument
from plottr.data import datadict_storage as dds, datadict as dd

## Configuration

In [ ]:
DATADIR = './data/'

## Create instruments

**TODO:** here we would prefer getting the instruments from the server.

In [ ]:
# Create a new station, close all previous instruments
Instrument.close_all()
station = Station()

In [ ]:
from instrumentserver.testing.dummy_instruments.rf import ResonatorResponse
vna = find_or_create_instrument(ResonatorResponse, 'vna')
station.add_component(vna)

vna.resonator_frequency(5e9)
vna.resonator_linewidth(10e6)

In [ ]:
from instrumentserver.testing.dummy_instruments.rf import FluxControl
flux = find_or_create_instrument(FluxControl, 'flux', vna)
station.add_component(flux)

# Example: Flux tuning a resonator

## Testing

In [ ]:
# just acquire a trace and plot. The resonance frequency should change when changing the flux.
vna.start_frequency(4e9)
vna.stop_frequency(5.2e9)
vna.npoints(1201)
vna.bandwidth(1e3)
vna.power(-50)
flux.flux(-0.25)

f_vals = vna.frequency()
s11_vals = vna.data()

fig, ax = plt.subplots(1, 1)
ax.plot(f_vals, np.angle(s11_vals))

## Measurement

### Basics

This is a very simple example for how to use DDH5Writer to save data to hdf5.
The basic structure is:

1. specify the structure of the data. This is required for two reasons: A, knowing the data structure before actually running measurement code allows to pre-define objects in the data files, which is important for accessing the data from multiple programs (single-writer multi-reader support). B, for all features in live-plotting and analysis to work we need to know the relations between different data objects (like dependent and independent data). 

2. use the Writer as context manager to add data (in practice, in some kind of measurement loop). 


### Live plotting

You can use `plottr` to live plot the data (or plot it later). An easy way to do this is by running the `monitr` app from the command line. This assumes that plottr is installed through pip, which should create the required launcher script. Run (and replace ``<datadir>`` with your data directory root):

``
    $ plottr-monitr <datadir>
``

You should see the monitr window open. On the left you will see all data files in your data directory. If you select a data file, the right hand side will display the contents of that file. you can plot data by right-clicking on a top-level group container and selecting 'Plot ``<group>``'.

If you activate the ``Auto-plot new`` button in the toolbar, any new data files appearing from this point on will be plotted automatically. The default refresh rate for the monitr app is 2 seconds (can be set a startup as a command line option).


### Notes

**TODO:** This is using the bare file writer. Some things, like setting up the data structure will be made easier, but the new measurement structures are will work in progress.

**TODO:** This isn't saving meta data yet, or copying additional files. An easy way to circumvent that for the time being is by inheriting from the DDH5 writer.

In [ ]:
# this is defining the data structure.
data = dd.DataDict(
    flux = dict(unit='Phi0'),
    frequency = dict(unit='Hz'),
    s11 = dict(axes=['flux', 'frequency']), # no unit, complex.
)
data.validate() # this is just for catching mistakes.

with dds.DDH5Writer(DATADIR, data, name='FluxSweep') as writer:
    for flux_val in np.linspace(-1,1,101):
        flux.flux(flux_val)
        time.sleep(0.2)
        
        # the writer accepts one line for each data field.
        # that means we should reshape the data that each entry has the form [<data>].
        # for practical purposes that means that each frequency and s11 data array
        # is like one datapoint.
        writer.add_data(
            flux = [flux_val],
            frequency = vna.frequency().reshape(1,-1),
            s11 = vna.data().reshape(1,-1),
        )
        
        

### Inspecting the data

This is just to illustrate how to open and look at the data. You often would not actually do that in the measurement notebook.

Note that the data is currently also in the memory. **TODO**: an operation mode for not keeping the data in the memory is not implemented yet.

To illustrate, we look at the data in the memory, and compare with the file.

In [ ]:
# when we look at the data now, we see it's not on a grid, even though we want the data to be.
pprint(data.structure())
pprint(data.shapes())

# this is detecting the grid in the data
data_as_grid = dd.datadict_to_meshgrid(data)
pprint(data_as_grid.structure())
pprint(data_as_grid.shapes())

flux_data = data_as_grid.data_vals('flux')
frq_data = data_as_grid.data_vals('frequency')
s11_data = data_as_grid.data_vals('s11')

fig, ax = plt.subplots(1, 1)

ax.imshow(
    np.angle(s11_data.T), 
    aspect='auto', origin='lower',
    extent=[flux_data.min(), flux_data.max(), frq_data.min(), frq_data.max()]
)
ax.set_xlabel('Flux (phi_0)')
ax.set_ylabel('Frequency (Hz)')

In [ ]:
# Loading from file is essentiall the same thing:
data_from_file = dds.datadict_from_hdf5(writer.file_path)
data_as_grid = dd.datadict_to_meshgrid(data_from_file)

flux_data = data_as_grid.data_vals('flux')
frq_data = data_as_grid.data_vals('frequency')
s11_data = data_as_grid.data_vals('s11')

fig, ax = plt.subplots(1, 1)

ax.imshow(
    np.angle(s11_data.T), 
    aspect='auto', origin='lower',
    extent=[flux_data.min(), flux_data.max(), frq_data.min(), frq_data.max()]
)
ax.set_xlabel('Flux (phi_0)')
ax.set_ylabel('Frequency (Hz)')